Imports

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
result1=pd.read_csv(r'Data\NEW_ADS3.csv')

In [3]:
train1 = result1[result1['data']=='train']
validation=result1[result1['data']=='submission_df']

In [4]:
y = train1['efficiency']
x = train1.drop(columns=['effectiveness_usd','efficiency','event_id', 'data','starttime'])


In [5]:
x.drop('finalspend_usd',axis=1,inplace=True)

Outlier Cap

In [6]:
# Calculate the IQR for X (independent variable)
Q1_X = x.quantile(0.25)
Q3_X = x.quantile(0.75)
IQR_X = Q3_X - Q1_X

# Calculate the IQR for Y (dependent variable)
Q1_Y = y.quantile(0.25)
Q3_Y = y.quantile(0.75)
IQR_Y = Q3_Y - Q1_Y

lower_bound_X = Q1_X - 1.5 * IQR_X
upper_bound_X = Q3_X + 1.5 * IQR_X

# Define the bounds for Y
lower_bound_Y = Q1_Y - 1.5 * IQR_Y
upper_bound_Y = Q3_Y + 1.5 * IQR_Y

In [7]:
# Cap the outliers for X
for column in x.columns:
    x[column] = np.where(x[column] < lower_bound_X[column], lower_bound_X[column], x[column])
    x[column] = np.where(x[column] > upper_bound_X[column], upper_bound_X[column], x[column])


In [8]:
y_capped = np.where(y < lower_bound_Y, lower_bound_Y, y)
y_capped = np.where(y_capped > upper_bound_Y, upper_bound_Y, y_capped)

# Convert back to Series if needed
y= pd.Series(y_capped, index=y.index)


RFE FS

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
# Split data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Standardize the features
scaler_x = StandardScaler()
x_train_scaled = scaler_x.fit_transform(x_train)
x_test_scaled = scaler_x.transform(x_test)

# Standardize the target variable
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))
from sklearn.feature_selection import RFE
model_rfe = LinearRegression()
selector_rfe = RFE(model_rfe, n_features_to_select=30)  # Specify number of features to keep
selector_rfe.fit(x_train_scaled, y_train_scaled)

# Get the selected features
selected_features = x_train.columns[selector_rfe.support_]
print("Selected Features by RFE:", selected_features)


Selected Features by RFE: Index(['Unnamed: 0', 'baselinespend_usd', 'bestbid_usd', 'participant',
       'itemno', 'ratio', 'biddingperiod', 'overtimeperiod(minutes)',
       'reviewtimeperiod', 'timebetweenlotclosing', 'is_weekend', 'item_name',
       'zone_APAC', 'zone_EUR', 'zone_MAZ', 'zone_NAZ', 'zone_SAZ',
       'event_template_ABI Reverse Auction (Formula)',
       'event_template_ABI Reverse Auction (Formula) - NI',
       'event_template_ABI Reverse Auction - NI',
       'canparticipantsplacebidsduringpreviewperiod_Do not allow prebids',
       'canparticipantsseeranks?_Their own rank',
       'improvebidamountby_Percentage',
       'setareviewperiodafterlotoritemcloses_Yes', 'Mean_Lot_Item_Bid_Version',
       'Count_supplier', 'Count_Item_Name', 'Count_Lot_Item_Bid_Version',
       'Count_Accepted', 'Average_savings'],
      dtype='object')


XGB FS

In [10]:
# Split data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Standardize the features
scaler_x = StandardScaler()
x_train_scaled = scaler_x.fit_transform(x_train)
x_test_scaled = scaler_x.transform(x_test)

# Train an XGBoost Model
xgb_model = XGBRegressor(random_state=42)
xgb_model.fit(x_train_scaled, y_train)

# Get feature importances
feature_importances = pd.Series(xgb_model.feature_importances_, index=x.columns)

# Select top n features
n_features = 35
selected_features = feature_importances.nlargest(n_features).index

print("Selected Features by XGBoost:", selected_features)

# Filter data to include only selected features
x_train_selected = x_train[selected_features]
x_test_selected = x_test[selected_features]


Selected Features by XGBoost: Index(['zone_APAC', 'categ_l2_IND CAPEX', 'Unnamed: 0',
       'overtimeperiod(minutes)', 'event_template_ABI Reverse Auction - NI',
       'ratio', 'canparticipantsseeranks?_Their own rank',
       'Mean_Lot_Item_Bid_Version', 'Average_savings', 'bestbid_usd',
       'Count_supplier', 'enabletrafficlightbidding_Yes', 'baselinespend_usd',
       'improvebidamountby_Percentage', 'bidrankthattriggersovertime',
       'Count_Lot_Item_Bid_Version',
       'showlineitemlevelrankinlot_Yes, to Buyers and Participants', 'itemno',
       'canparticipantsplacebidsduringpreviewperiod_Do not allow prebids',
       'canparticipantsseeranks?_No', 'biddingperiod',
       'setareviewperiodafterlotoritemcloses_Yes', 'participant',
       'reviewtimeperiod', 'timebetweenlotclosing', 'is_weekend', 'item_name',
       'zone_EUR', 'zone_MAZ', 'zone_NAZ', 'zone_SAZ',
       'event_template_ABI Reverse Auction (Formula)',
       'event_template_ABI Reverse Auction (Formula) - NI

In [11]:
additional_features = [
    'dag_categ_INDUSTRIAL CAPEX',
    'dag_categ_INDUSTRIAL CAPEX - GLOBAL - ABOVE 500K',
    'dag_categ_INDUSTRIAL CAPEX - LOCAL',
    'dag_categ_Indirect Trade Marketing - Distributors and Associations',
    'dag_categ_Office Supplies', 'dag_categ_Other Office Utilities',
    'dag_categ_Water, Electricity & Gas (Office)', 'categ_l1_COMMERCIAL',
    'categ_l1_PACKAGING', 'categ_l2_IND CAPEX', 'categ_l2_MARKETING',
    'categ_l2_OFFICE UTILITIES', 'categ_l2_PACKAGING', 'categ_l2_POCM',
    'categ_l2_SALES', 'baselinespend_usd', 'bestbid_usd', 'participant',
    'itemno', 'biddingperiod', 'timebetweenlotclosing',
    'setareviewperiodafterlotcloses_Yes', 'zone_APAC', 'zone_EUR', 'zone_MAZ',
    'zone_NAZ', 'zone_SAZ', 'enabletrafficlightbidding_Yes','ratio','is_weekend','Mean_Lot_Item_Bid_Version','Count_supplier','Count_Item_Name','Count_Lot_Item_Bid_Version','Count_Accepted','Average_savings']

selected_features = selected_features.tolist() if isinstance(selected_features, (pd.Series, np.ndarray)) else selected_features
additional_features = additional_features.tolist() if isinstance(additional_features, (pd.Series, np.ndarray)) else additional_features

selected_features = list(selected_features)

selected_features = selected_features+additional_features
selected_features = list(set(selected_features))

In [12]:
x=x[selected_features]

In [13]:
# x=x[['dag_categ_INDUSTRIAL CAPEX',
#        'dag_categ_INDUSTRIAL CAPEX - GLOBAL - ABOVE 500K',
#        'dag_categ_INDUSTRIAL CAPEX - LOCAL',
#        'dag_categ_Indirect Trade Marketing - Distributors and Associations',
#        'dag_categ_Office Supplies', 'dag_categ_Other Office Utilities',
#        'dag_categ_Water, Electricity & Gas (Office)', 'categ_l1_COMMERCIAL',
#        'categ_l1_PACKAGING', 'categ_l2_IND CAPEX', 'categ_l2_MARKETING',
#        'categ_l2_OFFICE UTILITIES', 'categ_l2_PACKAGING', 'categ_l2_POCM',
#        'categ_l2_SALES','baselinespend_usd','bestbid_usd','participant','itemno','biddingperiod','timebetweenlotclosing','setareviewperiodafterlotcloses_Yes','zone_APAC','zone_EUR','zone_MAZ','zone_NAZ','zone_SAZ','enabletrafficlightbidding_Yes']]

In [14]:
# x=x[['dag_categ_Corporate Sponsorship', 'dag_categ_Courier/ Fulfillment',
#        'dag_categ_Flavors',
#        'dag_categ_Fridges & Coolers-New (Global Supplier+Over $1M)',
#        'dag_categ_Heat - Heating oil', 'dag_categ_INDUSTRIAL CAPEX',
#        'dag_categ_INDUSTRIAL CAPEX - GLOBAL - ABOVE 500K',
#        'dag_categ_INDUSTRIAL CAPEX - LOCAL', 'dag_categ_IT CAPEX',
#        'dag_categ_IT CAPEX (Global / Cross Region)',
#        'dag_categ_IT CAPEX (Regional / Local)',
#        'dag_categ_Indirect Trade Marketing - Distributors and Associations',
#        'dag_categ_Legally Required Communication', 'dag_categ_Ocean',
#        'dag_categ_Office Supplies', 'dag_categ_Other Office Utilities',
#        'dag_categ_Plastic Labels', 'dag_categ_Postage/Courier',
#        'dag_categ_RETURNABLE PALLETS, CRATES',
#        'dag_categ_Water, Electricity & Gas (Office)', 'categ_l1_COMMERCIAL',
#        'categ_l1_PACKAGING', 'categ_l2_IND CAPEX', 'categ_l2_IT CAPEX',
#        'categ_l2_MARKETING', 'categ_l2_OFFICE UTILITIES', 'categ_l2_PACKAGING',
#        'categ_l2_POCM', 'categ_l2_SALES', 'categ_l2_SECONDARY PACKAGING','baselinespend_usd','bestbid_usd','participant','itemno','biddingperiod','timebetweenlotclosing','setareviewperiodafterlotcloses_Yes','zone_APAC','zone_EUR','zone_MAZ','zone_NAZ','zone_SAZ','enabletrafficlightbidding_Yes']]